In [3]:
import pandas as pd 
import numpy as np 
# Lectura de los datos 2D data_2d =
pd.read_csv("data_2d.csv") 
print("Datos 2D:") 
print(data_2d.head())

Datos 2D:


NameError: name 'data_2d' is not defined